<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install -q gputil
!pip install -q psutil
!pip install -q humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
process = psutil.Process(os.getpid())
print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
print("GPU Model: ", gpu.name)

Gen RAM Free: 12.7 GB  | Proc size: 117.9 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB
GPU Model:  Tesla K80


# Возьмем за основу третий урок (распознавание марок машин)  
В этом задании поставим задачу подобрать такую архитектуру сети, чтобы по качеству она была не хуже, как минимум, а по скорости быстрее той, которая была в третьем уроке.

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential #Сеть прямого распространения
#Базовые слои для счёрточных сетей
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator # работа с изображениями
from tensorflow.keras.optimizers import Adam, Adadelta # оптимизаторы
from tensorflow.keras import utils #Используем дял to_categoricall
from tensorflow.keras.preprocessing import image #Для отрисовки изображений
from google.colab import files #Для загрузки своей картинки
import numpy as np #Библиотека работы с массивами
import matplotlib.pyplot as plt #Для отрисовки графиков
from PIL import Image #Для отрисовки изображений
import random #Для генерации случайных чисел 
import math # Для округления
import os #Для работы с файлами 
# подключем диск
from google.colab import drive
from tensorflow.keras.callbacks import ModelCheckpoint
import time

%matplotlib inline

In [ ]:
!unzip "/content/drive/MyDrive/Neural_Univesity/Автомобили.zip" -d /content/cars #Указываем путь к базе в Google Drive

In [4]:
os.listdir('cars')

['Автомобили']

In [5]:
train_path = '/content/cars/Автомобили' #Папка с папками картинок, рассортированных по категориям
batch_size = 8 #Размер выборки
img_width = 96 #Ширина изображения
img_height = 54 #Высота изображения

In [6]:
#Генератор изображений
datagen = ImageDataGenerator(
    rescale=1. / 255, #Значения цвета меняем на дробные показания
    rotation_range=10, #Поворачиваем изображения при генерации выборки
    width_shift_range=0.1, #Двигаем изображения по ширине при генерации выборки
    height_shift_range=0.1, #Двигаем изображения по высоте при генерации выборки
    zoom_range=0.1, #Зумируем изображения при генерации выборки
    horizontal_flip=True, #Включаем отзеркаливание изображений
    fill_mode='nearest', #Заполнение пикселей вне границ ввода
    validation_split=0.2 #Указываем разделение изображений на обучающую и тестовую выборку
)

In [7]:
# обучающая выборка
train_generator = datagen.flow_from_directory(
    train_path, #Путь ко всей выборке выборке
    target_size=(img_width, img_height), #Размер изображений
    batch_size=batch_size, #Размер batch_size
    class_mode='categorical', #Категориальный тип выборки. Разбиение выборки по маркам авто 
    shuffle=True, #Перемешивание выборки
    subset='training' # устанавливаем как набор для обучения
)

# проверочная выборка
validation_generator = datagen.flow_from_directory(
    train_path, #Путь ко всей выборке выборке
    target_size=(img_width, img_height), #Размер изображений
    batch_size=batch_size, #Размер batch_size
    class_mode='categorical', #Категориальный тип выборки. Разбиение выборки по маркам авто 
    shuffle=True, #Перемешивание выборки
    subset='validation' # устанавливаем как валидационный набор
)

Found 2743 images belonging to 2 classes.
Found 684 images belonging to 2 classes.


**Входной слой**

*   0 - делаем ли нормализацию
*   1 - размер первого свёрточного слоя
*   2 - ядро первого свёрточного слоя 
*   3 - функция активации первого слоя
*   4 - делаем ли MaxPooling0
*   5 - размер MaxPooling0

**Первый скрытый слой**

*   6 - Делаем ли второй сверточный слой
*   7 - размер второго сверточного слоя
*   8 - ядро второго сверточного слоя
*   9 - делаем ли MaxPooling1
*   10 - размер MaxPooling1
*   11 - функция активации

**Второй скрытый слой**

*   12 - Делаем ли третий сверточный слой
*   13 - размер третьего сверточного слоя
*   14 - ядро третьего сверточного слоя
*   15 - делаем ли MaxPooling2
*   16 - размер MaxPooling2
*   17 - функция активации 

**Третий скрытый слой**

*   18 - Делаем ли третий сверточный слой
*   19 - размер третьего сверточного слоя
*   20 - ядро третьего сверточного слоя
*   21 - делаем ли MaxPooling2
*   22 - размер MaxPooling2
*   23 - функция активации 

**Четвертый скрытый слой**

*   24 - Делаем ли четвертый сверточный слой
*   25 - размер четвертого сверточного слоя
*   26 - ядро четвертого сверточного слоя
*   27 - делаем ли MaxPooling2
*   28 - размер MaxPooling2
*   29 - функция активации 

**Пятый скрытый слой**

*   30 - Делаем ли пятый сверточный слой
*   31 - размер пятого сверточного слоя
*   32 - ядро пятого сверточного слоя
*   33 - делаем ли MaxPooling2
*   34 - размер MaxPooling2
*   35 - функция активации 

**Шестой скрытый слой**
*   36 - делаем ли шестой полносвязный слой
----
*   37 - функция активации шестого слоя
*   38 - функция активации предпоследнего слоя
*   39 - функция активации выходного слоя
----
**Шестой скрытый слой**

*   40 - делаем ли нормализацию 
*   41 - размер полносвязного слоя

**Седьмой (предпоследний) скрытый слой**

*   42 - делаем ли нормализацию 
*   43 - размер полносвязного слоя

### Задумка была такая, но колаб не даёт считать долго, поэтому от части параметров пришлось отказаться.

In [8]:
#Создаём сеть (net - список параметров)
def createConvNet(net):

  model = Sequential()             # Создаем моель Sequential
  
  makeFirstNormalization = net[0]  # Делаем ли нормализацию в начале
  firstConvSize = 2 ** net[1]      # Размер входного вёрточного слоя
  firstConvKernel = net[2]         # Ядро входного свёрточного слоя
  activation0 = net[3]             # Функция активации входного слоя
  makeMaxPooling0 = net[4]         # Делаем ли maxpooling для нулевого слоя
  maxPoolingSize0 = net[5]         # Размер MaxPooling

  makeSecondConv = net[6]          # Делаем ли первый свёрточный слой
  secondConvSize = 2 ** net[7]     # Размер первого свёрточного слоя
  secondConvKernel = net[8]        # Ядро второго свёрточного слоя
  makeMaxPooling1 = net[9]         # Делаем ли MaxPooling
  maxPoolingSize1 = net[10]        # Размер MaxPooling
  activation1 = net[11]            # Функция активации

  makeThirdConv = net[12]          # Делаем ли второй свёрточный слой
  thirdConvSize = 2 ** net[13]     # Размер второго свёрточного слоя
  thirdConvKernel = net[14]        # Ядро второго свёрточного слоя
  makeMaxPooling2 = net[15]        # Делаем ли MaxPooling
  maxPoolingSize2 = net[16]        # Размер MaxPooling
  activation2 = net[17]            # Функция активации
  
#   makeFourthConv = net[18]          # Делаем ли третий свёрточный слой
#   fourthConvSize = 2 ** net[19]     # Размер тертьего свёрточного слоя
#   fourthConvKernel = net[20]        # Ядро третьего свёрточного слоя
#   makeMaxPooling3 = net[21]        # Делаем ли MaxPooling
#   maxPoolingSize3 = net[22]        # Размер MaxPooling
#   activation3 = net[23]            # Функция активации

#   makeFifthConv = net[24]          # Делаем ли четвертый свёрточный слой
#   fifthConvSize = 2 ** net[25]     # Размер четвертого свёрточного слоя
#   fifthConvKernel = net[26]        # Ядро четвертго свёрточного слоя
#   makeMaxPooling4 = net[27]        # Делаем ли MaxPooling
#   maxPoolingSize4 = net[28]        # Размер MaxPooling
#   activation4 = net[29]            # Функция активации

#   makeSixthhConv = net[30]          # Делаем ли пятый свёрточный слой
#   sixthConvSize = 2 ** net[31]     # Размер пятого свёрточного слоя
#   sixthConvKernel = net[32]        # Ядро пятого свёрточного слоя
#   makeMaxPooling5 = net[33]        # Делаем ли MaxPooling
#   maxPoolingSize5 = net[34]        # Размер MaxPooling
#   activation5 = net[35]            # Функция активации

  makeSeventhDense = net[18]          # Делаем ли шестой полносвязгый слой

  activation6 = net[19]            # Функция активации для шестого слоя
  activation7 = net[20]            # Функция активации для предпоследнего
  activation8 = net[21]            # Функция активации для последнего слоя

  makeSeventhNormalization = net[22] # Делаем ли нормализацию перед шестым слоем
  seventhdenseSize = 2 ** net[23]          # Размер шестого полносвязного слоя

  makeFinalNormalization = net[24] # Делаем ли финальную нормализацию
  denseSize = 2 ** net[25]          # Размер полносвязного слоя

  activation_list = ['linear','relu','tanh','softmax','sigmoid'] 

  if (makeFirstNormalization):      # Если делаем нормализацию в начале
    # Добавляем слой BatchNormalization
    model.add(BatchNormalization(input_shape=(img_width, img_height, 3)))
    # Добавляем Conv1D-слой с firstConvSize нейронами и ядром (firstConvKernel)
    model.add(Conv2D(firstConvSize, firstConvKernel, activation=activation_list[activation0], padding='same')) 
  else:
    # Добавляем Conv1D-слой с firstConvSize нейронами и ядром (firstConvKernel)
    model.add(Conv2D(firstConvSize, firstConvKernel, input_shape=(img_width, img_height, 3), activation=activation_list[activation0], padding='same')) 
  
  if makeMaxPooling0:               # Если делаем maxpooling
    model.add(MaxPooling2D(maxPoolingSize0))

  if (makeSecondConv):              # Если делаем первый свёрточный слой
    # Добавляем Conv2D-слой с secondConvSize нейронами и ядром (secondConvKernel)
    model.add(Conv2D(secondConvSize, secondConvKernel, activation=activation_list[activation1], padding='same')) 
    
    if (makeMaxPooling1):           # Если делаем MaxPooling
      # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
      model.add(MaxPooling2D(pool_size=maxPoolingSize1)) 

  if (makeThirdConv):              # Если делаем второй свёрточный слой
    # Добавляем Conv2D-слой с secondConvSize нейронами и ядром (secondConvKernel)
    model.add(Conv2D(thirdConvSize, thirdConvKernel, activation=activation_list[activation2], padding='same')) 
    
    if (makeMaxPooling2):           # Если делаем MaxPooling
      # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
      model.add(MaxPooling2D(pool_size=maxPoolingSize2)) 

#   if (makeFourthConv):              # Если делаем третий свёрточный слой
#     # Добавляем Conv2D-слой с secondConvSize нейронами и ядром (secondConvKernel)
#     model.add(Conv2D(fourthConvSize, fourthConvKernel, activation=activation_list[activation3], padding='same')) 
    
#     if (makeMaxPooling3):           # Если делаем MaxPooling
#       # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
#       model.add(MaxPooling2D(pool_size=maxPoolingSize3)) 

#   if (makeFifthConv):              # Если делаем четвертый свёрточный слой
#     # Добавляем Conv2D-слой с secondConvSize нейронами и ядром (secondConvKernel)
#     model.add(Conv2D(fifthConvSize, fifthConvKernel, activation=activation_list[activation4], padding='same')) 
    
#     if (makeMaxPooling4):           # Если делаем MaxPooling
#       # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
#       model.add(MaxPooling2D(pool_size=maxPoolingSize4)) 

#   if (makeSixthhConv):              # Если делаем пятый свёрточный слой
#     # Добавляем Conv2D-слой с secondConvSize нейронами и ядром (secondConvKernel)
#     model.add(Conv2D(sixthConvSize, sixthConvKernel, activation=activation_list[activation5], padding='same')) 
    
#     if (makeMaxPooling5):           # Если делаем MaxPooling
#       # Добавляем слой MaxPooling1D с размером (maxPoolingSize)
#       model.add(MaxPooling2D(pool_size=maxPoolingSize5)) 

  if (makeSeventhNormalization):
    #   добавляем нормализацию
    model.add(BatchNormalization())

  model.add(Flatten())              # Добавляем слой Flatten
  
  if (makeSeventhDense):
      model.add(Dense(seventhdenseSize, activation=activation_list[activation6]))
  
  if (makeFinalNormalization):
    #   добавляем нормализацию
    model.add(BatchNormalization())

  model.add(Dense(denseSize, activation=activation_list[activation7])) # Добавляем слой Dense с denseSize нейронами
  model.add(Dense(len(train_generator.class_indices), activation=activation_list[activation8]))

  return model                      # Возвращаем модель


In [9]:
def evaluateNet(net, ep, verb):
    '''
    Функция вычисления результата работы сети
    '''
    val = 0
    start = time.time()
    model = createConvNet(net) # Создаем модель createConvNet

    # Компилируем модель
    model.compile(optimizer=Adam(learning_rate=1e-4),
                    loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
            train_generator,
            steps_per_epoch = train_generator.samples // batch_size,
            validation_data = validation_generator, 
            validation_steps = validation_generator.samples // batch_size,
            epochs=5,
            verbose=verb,
        )
        
    val = history.history["val_accuracy"][-1] - float(time.time() - start) / 1000 # Возвращаем точность на проверочной выборке с последней эпохи минус время, для наиболее точной и быстрой нейронки
    
    return val, model                      # Возвращаем точность

In [10]:

def createRandomNet():
    '''
    Функция создания списка случайных параметров
    '''
    net = []
    net.append(random.randint(0,1)) #Делаем или нет нормализацию
    net.append(random.randint(3,8)) #Первый свёрточный слой от 8 до 64 нейронов
    net.append(random.randint(3,7)) #Ядро первого свёрточного слоя от 3 до 7
    net.append(random.randint(0,4)) # Функция активации входного слоя
    net.append(random.randint(0,1)) #Делаем ли MaxPooling
    net.append(random.randint(2,3)) #Размер MaxPooling

    net.append(random.randint(0,1)) # Сколько делаем еще сверточных слоев
    net.append(random.randint(3,8)) # Второй свёрточный слой от 8 до 64 нейронов
    net.append(random.randint(3,7)) # Ядро первого свёрточного слоя от 3 до 7
    net.append(random.randint(0,1)) # Делаем ли MaxPooling
    net.append(random.randint(2,3)) # Размер MaxPooling
    net.append(random.randint(0,4)) # Функция активации первого слоя

    net.append(random.randint(0,1)) # Сколько делаем еще сверточных слоев
    net.append(random.randint(3,8)) # Второй свёрточный слой от 8 до 64 нейронов
    net.append(random.randint(3,7)) # Ядро второго свёрточного слоя от 3 до 7
    net.append(random.randint(0,1)) # Делаем ли MaxPooling
    net.append(random.randint(2,3)) # Размер MaxPooling
    net.append(random.randint(0,4)) # Функция активации второго слоя

    # net.append(random.randint(0,1)) # Сколько делаем еще сверточных слоев
    # net.append(random.randint(3,10)) # Второй свёрточный слой от 8 до 64 нейронов
    # net.append(random.randint(3,7)) # Ядро третьего свёрточного слоя от 3 до 7
    # net.append(random.randint(0,1)) # Делаем ли MaxPooling
    # net.append(random.randint(2,3)) # Размер MaxPooling
    # net.append(random.randint(0,4)) # Функция активации третьего слоя

    # net.append(random.randint(0,1)) # Сколько делаем еще сверточных слоев
    # net.append(random.randint(3,10)) # Второй свёрточный слой от 8 до 64 нейронов
    # net.append(random.randint(3,7)) # Ядро четвертого свёрточного слоя от 3 до 7
    # net.append(random.randint(0,1)) # Делаем ли MaxPooling
    # net.append(random.randint(2,3)) # Размер MaxPooling
    # net.append(random.randint(0,4)) # Функция активации четвертого слоя

    # net.append(random.randint(0,1)) # Сколько делаем еще сверточных слоев
    # net.append(random.randint(3,10)) # Пятый свёрточный слой от 8 до 64 нейронов
    # net.append(random.randint(3,7)) # Ядро пятого свёрточного слоя от 3 до 7
    # net.append(random.randint(0,1)) # Делаем ли MaxPooling
    # net.append(random.randint(2,3)) # Размер MaxPooling
    # net.append(random.randint(0,4)) # Функция активации пятого слоя

    net.append(random.randint(0,1)) # Делаем или нет шестой полносвязный слой
    
    net.append(random.randint(0,4)) # Функция активации шестого dense слоя
    net.append(random.randint(0,4)) # Функция активации предпоследнего dense слоя
    net.append(random.randint(0,4)) # Функция активации последнего слоя

    net.append(random.randint(0,1)) # Делаем ли нормализацию  перед шестым слоем
    net.append(random.randint(3,10)) # Размер шестого полносвязного слоя от 8 до 64

    net.append(random.randint(0,1)) # Делаем ли финальную нормализацию слой
    net.append(random.randint(3,10)) # Размер полносвязного слоя от 8 до 64
    
    return net

In [11]:
len(createRandomNet())

26

In [12]:
n = 10              # Общее число ботов
nsurv = 5          # Количество выживших (столько лучших переходит в новую популяцию)
nnew = n - nsurv    # Количество новых (столько новых ботов создается)
l = 26              # Размер бота
epohs = 10          # количество эпох

mut = 0.2          # коэфициент мутаций

popul = []          # Массив популяции
val = []            # Одномерный массив значений этих ботов

#Создаём случайных ботов
for i in range(n):
  popul.append(createRandomNet())
  
for it in range(epohs):                 # Пробегаем по всем эпохам
  val = []                              # Обнуляем значения бота
  curr_time = time.time()
  for i in range(n):                    # Пробегаем в цикле по всем ботам 
    bot = popul[i]                      # Берем очередного бота
    f, model_sum = evaluateNet(bot, 5, 1) # Вычисляем точность текущего бота
    val.append(f)                       # Добавляем полученное значение в список val
  
  sval = sorted(val, reverse=1)         # Сортируем val
  # Выводим 5 лучших ботов
  print(it, time.time() - curr_time, " ", sval[0:5],popul[:5]) 
  
  newpopul = []                         # Создаем пустой список под новую популяцию
  for i in range(nsurv):                # Пробегаем по всем выжившим ботам
    index = val.index(sval[i])          # Получаем индекс очередного бота из списка лучших в списке val
    newpopul.append(popul[index])       # Добавляем в новую популяцию бота из popul с индексом index
    
  for i in range(nnew):                 # Проходимся в цикле nnew-раз  
    indexp1 = random.randint(0,nsurv-1) # Случайный индекс первого родителя в диапазоне от 0 до nsurv - 1
    indexp2 = random.randint(0,nsurv-1) # Случайный индекс первого родителя в диапазоне от 0 до nsurv - 1
    botp1 = newpopul[indexp1]           # Получаем первого бота-родителя по indexp1
    botp2 = newpopul[indexp2]           # Получаем второго бота-родителя по indexp2    
    newbot = []                         # Создаем пустой список под значения нового бота    
    net4Mut = createRandomNet()         # Создаем случайную сеть для мутаций
    for j in range(l):                  # Пробегаем по всей длине размерности (784)      
      x = 0      
      pindex = random.random()          # Получаем случайное число в диапазоне от 0 до 1

      # Если pindex меньше 0.5, то берем значения от первого бота, иначе от второго
      if pindex < 0.5:
        x = botp1[j]
      else:
        x = botp2[j]
      
      # С вероятностью mut устанавливаем значение бота из net4Mut
      if (random.random() < mut):
        x = net4Mut[j]
        
      newbot.append(x)                  # Добавляем очередное значение в нового бота      
    newpopul.append(newbot)             # Добавляем бота в новую популяцию      
  popul = newpopul                      # Записываем в popul новую посчитанную популяцию

Epoch 1/5
342/342 [==============================] - 44s 35ms/step - loss: 2.6899 - accuracy: 0.7689 - val_loss: 1.8581 - val_accuracy: 0.7868
Epoch 2/5
342/342 [==============================] - 12s 35ms/step - loss: 2.2315 - accuracy: 0.7839 - val_loss: 1.8197 - val_accuracy: 0.7941
Epoch 3/5
342/342 [==============================] - 12s 35ms/step - loss: 2.1773 - accuracy: 0.7876 - val_loss: 1.9606 - val_accuracy: 0.7926
Epoch 4/5
342/342 [==============================] - 12s 34ms/step - loss: 2.2177 - accuracy: 0.7828 - val_loss: 1.9302 - val_accuracy: 0.7794
Epoch 5/5
342/342 [==============================] - 12s 34ms/step - loss: 2.1490 - accuracy: 0.7850 - val_loss: 1.8516 - val_accuracy: 0.7956
Epoch 1/5
342/342 [==============================] - 32s 92ms/step - loss: 0.8431 - accuracy: 0.7678 - val_loss: 0.6392 - val_accuracy: 0.7324
Epoch 2/5
342/342 [==============================] - 32s 92ms/step - loss: 0.5644 - accuracy: 0.7751 - val_loss: 0.5823 - val_accuracy: 0.7794

In [15]:
model_new = createConvNet(popul[0])
model_new.summary()

Model: "sequential_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 96, 54, 8)         872       
_________________________________________________________________
max_pooling2d_120 (MaxPoolin (None, 48, 27, 8)         0         
_________________________________________________________________
batch_normalization_185 (Bat (None, 48, 27, 8)         32        
_________________________________________________________________
flatten_101 (Flatten)        (None, 10368)             0         
_________________________________________________________________
dense_294 (Dense)            (None, 32)                331808    
_________________________________________________________________
batch_normalization_186 (Bat (None, 32)                128       
_________________________________________________________________
dense_295 (Dense)            (None, 1024)           

In [17]:
#Комипилируем и обучаем
start = time.time()
model_new.compile(optimizer=Adam(learning_rate=1e-4),
                loss='categorical_crossentropy', metrics=['accuracy'])
history = model_new.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        validation_data = validation_generator, 
        validation_steps = validation_generator.samples // batch_size,
        epochs=20,
        verbose=1,
    )
print('Выполнено за ', time.time() - start)

Epoch 1/20
342/342 [==============================] - 32s 34ms/step - loss: 0.5431 - accuracy: 0.8007 - val_loss: 0.5350 - val_accuracy: 0.8015
Epoch 2/20
342/342 [==============================] - 12s 34ms/step - loss: 0.5326 - accuracy: 0.8000 - val_loss: 0.5247 - val_accuracy: 0.8000
Epoch 3/20
342/342 [==============================] - 12s 34ms/step - loss: 0.5243 - accuracy: 0.7996 - val_loss: 0.5187 - val_accuracy: 0.8015
Epoch 4/20
342/342 [==============================] - 12s 35ms/step - loss: 0.5170 - accuracy: 0.8004 - val_loss: 0.5131 - val_accuracy: 0.8000
Epoch 5/20
342/342 [==============================] - 11s 33ms/step - loss: 0.5133 - accuracy: 0.8004 - val_loss: 0.5092 - val_accuracy: 0.8015
Epoch 6/20
342/342 [==============================] - 11s 33ms/step - loss: 0.5079 - accuracy: 0.8004 - val_loss: 0.5091 - val_accuracy: 0.8000
Epoch 7/20
342/342 [==============================] - 12s 34ms/step - loss: 0.5066 - accuracy: 0.8007 - val_loss: 0.5048 - val_accuracy:

# Предыдущая архитектура

In [18]:
#Создаем последовательную модель
model = Sequential()
#Первый сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu', input_shape=(img_width, img_height, 3)))
#Второй сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
#Третий сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
#Слой регуляризации Dropout
model.add(Dropout(0.2))
#Четвертый сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
#Слой регуляризации Dropout
model.add(Dropout(0.2))
#Пятый сверточный слой
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
#Шестой сверточный слой
model.add(Conv2D(1024, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
#Слой регуляризации Dropout
model.add(Dropout(0.2))
#Слой преобразования двумерных данных в одномерные 
model.add(Flatten())
#Полносвязный слой
model.add(Dense(4096, activation='relu'))
#Полносвязный слой
model.add(Dense(2048, activation='relu'))
#Вызодной полносвязный слой
model.add(Dense(len(train_generator.class_indices), activation='softmax'))



In [19]:
#Комипилируем и обучаем
start = time.time()
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        validation_data = validation_generator, 
        validation_steps = validation_generator.samples // batch_size,
        epochs=20,
        verbose=1,
    )
print('Выполнено за ', time.time() - start)

Epoch 1/20
342/342 [==============================] - 41s 111ms/step - loss: 0.5218 - accuracy: 0.7978 - val_loss: 0.4998 - val_accuracy: 0.8029
Epoch 2/20
342/342 [==============================] - 36s 106ms/step - loss: 0.5061 - accuracy: 0.8004 - val_loss: 0.5018 - val_accuracy: 0.8000
Epoch 3/20
342/342 [==============================] - 36s 106ms/step - loss: 0.5032 - accuracy: 0.8007 - val_loss: 0.5130 - val_accuracy: 0.8000
Epoch 4/20
342/342 [==============================] - 36s 106ms/step - loss: 0.5041 - accuracy: 0.8004 - val_loss: 0.5020 - val_accuracy: 0.8015
Epoch 5/20
342/342 [==============================] - 36s 106ms/step - loss: 0.5012 - accuracy: 0.8007 - val_loss: 0.4968 - val_accuracy: 0.8015
Epoch 6/20
342/342 [==============================] - 36s 106ms/step - loss: 0.5030 - accuracy: 0.8004 - val_loss: 0.5005 - val_accuracy: 0.8000
Epoch 7/20
342/342 [==============================] - 36s 106ms/step - loss: 0.5011 - accuracy: 0.8000 - val_loss: 0.5093 - val_ac

In [20]:
model.summary()

Model: "sequential_102"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_189 (Conv2D)          (None, 96, 54, 256)       7168      
_________________________________________________________________
conv2d_190 (Conv2D)          (None, 96, 54, 256)       590080    
_________________________________________________________________
max_pooling2d_121 (MaxPoolin (None, 32, 18, 256)       0         
_________________________________________________________________
conv2d_191 (Conv2D)          (None, 32, 18, 256)       590080    
_________________________________________________________________
dropout (Dropout)            (None, 32, 18, 256)       0         
_________________________________________________________________
conv2d_192 (Conv2D)          (None, 32, 18, 256)       590080    
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 10, 6, 256)     

## Итог: при одинаковом качестве архитектура, подобранная с помощью генетического алгоритма, обучается в 2.5 раза быстрее имея всего 368 682 параметров против 41 241 858